In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import matplotlib.pyplot as plt
import numpy as np
from scipy import fft
from scipy import signal
import sklearn as sk
from sklearn import preprocessing
#from ipynb.fs.full.ToolBox import connect_to_db

In [3]:
engine = connect_to_db()

Good to go.


In [4]:
# Returns a dataframe of the longest modec (flight level) == cfl (cleared flight level) plateau, filters out null values
def isolate_plateau(ifpsid):
    df = pd.read_sql_query("""SELECT instant,tas,plateau_cfl,cflok FROM sbx1.<table> WHERE ifpsid = '{ii}'""".format(ii=ifpsid),engine)
    df = df[df.tas.notna() & df.cflok == 1]
    df = df.loc[df.plateau_cfl == df.groupby('plateau_cfl').instant.apply(np.ptp).idxmax()]
    return df

# Applies a median filter to tas (true air speed) and picks out every nth value
def medfilt(df,ks=5):
    return signal.medfilt(df.tas,kernel_size=ks)[2::ks]


In [373]:
%%time
## construction of hist feature vectors
# Note: Will not work without database access. Use df_hist_l2_12bins.pickle for further processing.
df_ifpsid = pd.read_pickle("../Notebooks/pickles/df_ifpsid_suitable.pickle")
for mode in ['l1','l2','max']:
    print("Starting "+mode)
    d_list = []
    bin_n = 12
    for index, row in df_ifpsid.iterrows():
        df = pd.read_sql_query("""SELECT instant,tas,plateau_cfl,cflok FROM sbx1.<table> WHERE ifpsid = '{ii}'""".format(ii=row.ifpsid),engine)
        df_iso = isolate_plateau(row.ifpsid)
        med = medfilt(df_iso)
        seq_avg = 0.5*(min(med)+max(med))
        step = seq_avg*0.05/(bin_n-2)
        #upper = seq_avg*1.025
        lower = seq_avg*0.975
        b = [min(lower,min(med))]+[seq_avg*0.975+x*step for x in range(bin_n-1)]+[max(seq_avg*0.975+(bin_n-2)*step,max(med))]
        hist = np.histogram(a=med,bins=b)
        vector = preprocessing.normalize([hist[0]],norm=mode).flatten()
        d_data_f = {'h'+str(k): vector[k] for k in range(bin_n)}
        d_data_i = {'ifpsid': row.ifpsid}
        d_list.append({**d_data_i, **d_data_f})
    df_hist = pd.DataFrame.from_dict(d_list)
    df_hist.to_pickle(path="../Notebooks/pickles/df_hist_{md}_{n}bins.pickle".format(md=mode,n=str(bin_n)))

Starting l1
Starting l2
Starting max
Wall time: 2h 37min 38s


In [2]:
pd.read_pickle("../pickles/df_hist_l2_12bins.pickle").sample(10)

,ifpsid,h0,h1,h2,h3,h4,h5,h6,h7,h8,h9,h10,h11
51145,AA15515021,0.0,0.0,0.000000,0.030744,0.184463,0.676363,0.491900,0.491900,0.153719,0.000000,0.0,0.0
88591,AA17246463,0.0,0.0,0.000000,0.108786,0.108786,0.108786,0.707107,0.598321,0.326357,0.000000,0.0,0.0
150296,AA16562191,0.0,0.0,0.060193,0.180579,0.240772,0.260836,0.902894,0.120386,0.080257,0.040129,0.0,0.0
1480,AA15398822,0.0,0.0,0.000000,0.000000,0.012306,0.110757,0.984508,0.135370,0.000000,0.000000,0.0,0.0
3266,AA15336774,0.0,0.0,0.000000,0.000000,0.242313,0.605783,0.757228,0.030289,0.000000,0.000000,0.0,0.0
80110,AA16560269,0.0,0.0,0.000000,0.096169,0.625101,0.673186,0.360635,0.120212,0.048085,0.000000,0.0,0.0
77902,AA15843262,0.0,0.0,0.138675,0.000000,0.208013,0.346688,0.277350,0.554700,0.624038,0.208013,0.0,0.0
95806,AA15759635,0.0,0.0,0.000000,0.000000,0.000000,0.196116,0.980581,0.000000,0.000000,0.000000,0.0,0.0
171348,AA15968159,0.0,0.0,0.000000,0.000000,0.000000,0.358979,0.933346,0.000000,0.000000,0.000000,0.0,0.0
8908,AA15956965,0.0,0.0,0.000000,0.000000,0.000000,0.066519,0.997785,0.000000,0.000000,0.000000,0.0,0.0
